In [1]:
import json

data = []
with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
    data = f.readlines()

json_lines = []

for line in data:
    jsline = json.loads(line)
    json_lines.append(jsline)

In [51]:
import pandas as pd

frame = pd.DataFrame.from_dict(json_lines)

In [52]:
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
del value['trackeeHistory']

# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
del frame['regionsNodesIds']
del frame['error']
del frame['type']

In [76]:
frame = frame.sort_values(by='measurementTimestamp')


,measurementTimestamp,processingTimestamp,sourceMac,coordinates
73,1436047367297,1436047381262,62e72aeb-2c41-44ed-88a6-3423267c1cb5,"[-7.459665, -75.723003, 0.0]"
141,1436047367330,1436047381441,7c824b77-a38b-4c3d-bb6a-3e40f5373d97,"[46.207027, -29.514564, 0.0]"
149,1436047367683,1436047381455,4ff97883-b9e8-4e2e-8b22-9bd093cfa401,"[-110.189892, -2.635529, 0.0]"


In [87]:
timewindow = frame[frame['measurementTimestamp'] < min(frame['measurementTimestamp']) + 180000]

In [89]:
len(window)

3755

In [6]:
# get most frequent mac address
macs = frame['sourceMac'].value_counts()
macs[:5]

2d1ff804-c499-4163-b90f-003df1e4ec65    78664
85a84e08-c967-41d0-b56b-522574796776    74356
2c019c2d-7e9e-42b1-b3bd-1797c918d4a7    16995
e1deda99-163f-4b68-9ade-e1f05d070bf2     8370
6cab229a-e15d-48b7-ab1e-5ddca3d7e283     7712
Name: sourceMac, dtype: int64

In [60]:
len(macs)

30344

In [173]:
from math import floor, ceil
import numpy as np

width = 80; height = 60; periods = 100 # was 10

bins = np.zeros((periods,height,width))

interval = 180000 # milliseconds
shift = 10000

for k in range(periods):
    #start = min(frame['measurementTimestamp']) + k * interval
    start = min(frame['measurementTimestamp']) + k * shift
    stop =  min(frame['measurementTimestamp']) + k * shift + interval
    #stop = min(frame['measurementTimestamp']) + (k+1) * interval
    timewindow = frame[frame['measurementTimestamp'] < stop]
    timewindow = timewindow[timewindow['measurementTimestamp'] >= start]
    
    normalization = 0
    
    #bins.append()

    for i in range(300): # len(macs)
        # filter for mac address
        mac_filtered_frame = timewindow[timewindow['sourceMac'] == macs.index[i]]
        # extract positions
        locations = mac_filtered_frame['coordinates']

        positions1 = pd.DataFrame(locations)
        positions = pd.DataFrame(list(positions1['coordinates']))

        f = lambda x: x/3

        # binned positions, still fractional numbers
        binpos = positions.apply(f)

        for j in range(len(binpos) - 1):
            if floor(binpos[0][j]) >= -width/2 and floor(binpos[0][j]) < width/2 and \
                floor(binpos[1][j]) >= -height/2 and floor(binpos[1][j]) < height/2:
                bins[k][height/2 + floor(binpos[1][j])][width/2 + floor(binpos[0][j])] += 1
                normalization += 1
            
    bins[k] /= normalization

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:39: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [122]:
bins[2].sum()

1.0

In [136]:
bins.shape

(10, 60, 80)

In [137]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure(figsize=(12,8))

X = np.arange(0,width,1)
Y = np.arange(0,height,1)
Xs, Ys = np.meshgrid(X, Y)

Z = bins[0]

ax = Axes3D(fig)
ax.plot_surface(Xs, Ys, Z, rstride=2, cstride=1, cmap='hsv')#, cmap='hot')
plt.show()

In [174]:
im = plt.imread('/Users/philip/Documents/PhD/data-analysis/escience/arena_sensation.png')

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
# convenience method:
fig, ax = plt.subplots(figsize=(12,8))

#ax.set_xlim((0,1853)) # 1853
#ax.set_ylim((0,1369)) # 1369

ax.imshow(im)

for i in range(100):
    ax.imshow(bins[i], aspect='auto', cmap='hsv', alpha=1)
    if i < 10:
        number = '000' + str(i)
    else:
        number = '00' + str(i)
    plt.savefig('probability-map%s.png' % number)
    
#plt.show()

In [167]:
number

'0009'